In [1]:
import os
import numpy as np
#import pandas as pd
os.chdir("../input/word2vec-nlp-tutorial")

In [2]:
!unzip -o labeledTrainData.tsv.zip -d /kaggle/working
!unzip -o testData.tsv.zip -d /kaggle/working
!unzip -o unlabeledTrainData.tsv.zip -d /kaggle/working

Archive:  labeledTrainData.tsv.zip
  inflating: /kaggle/working/labeledTrainData.tsv  
Archive:  testData.tsv.zip
  inflating: /kaggle/working/testData.tsv  
Archive:  unlabeledTrainData.tsv.zip
  inflating: /kaggle/working/unlabeledTrainData.tsv  


In [3]:
os.chdir('/kaggle/working')
with open('labeledTrainData.tsv','r',encoding='utf-8') as f1:
    trdata = f1.readlines()
    trlist = []
    trrate = []
    for sent in trdata[1:]:
        sen = sent.strip('\n').split('\t')
        trlist.append(sen[2].split(' '))
        trrate.append(sen[1])
    print(trlist[0])

['"With', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'MJ', "i've", 'started', 'listening', 'to', 'his', 'music,', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there,', 'watched', 'The', 'Wiz', 'and', 'watched', 'Moonwalker', 'again.', 'Maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent.', 'Moonwalker', 'is', 'part', 'biography,', 'part', 'feature', 'film', 'which', 'i', 'remember', 'going', 'to', 'see', 'at', 'the', 'cinema', 'when', 'it', 'was', 'originally', 'released.', 'Some', 'of', 'it', 'has', 'subtle', 'messages', 'about', "MJ's", 'feeling', 'towards', 'the', 'press', 'and', 'also', 'the', 'obvious', 'message', 'of', 'drugs', 'are', 'bad', "m'kay.<br", '/><br', '/>Visually', 'impressive', 'but', 'of', 'course', 'this', 'is', 'al

In [4]:

with open('testData.tsv','r',encoding='utf-8') as f1:
    tedata = f1.readlines()
    telist = []
    for sent in tedata[1:]:
        sen = sent.strip('\n').split('\t')
        telist.append(sen[1].split(' '))
with open('unlabeledTrainData.tsv','r',encoding='utf-8') as f1:
    tr_nolabel = f1.readlines()
    tr_nolist = []
    for sent in tr_nolabel[1:]:
        sen = sent.strip('\n').split('\t')
        tr_nolist.append(sen[1].split(' '))

In [5]:
import argparse
from gensim.models import word2vec
from time import time

path_prefix = './'
worddata = trlist+telist+tr_nolist
print("the len of data is {}".format(len(worddata)))
st=time()
model = word2vec.Word2Vec(worddata,vector_size=250, window=5, min_count=5, workers=12, epochs=10, sg=0)
model.save(os.path.join(path_prefix, 'w2v_all.model'))
print("we use time {} min".format((time()-st)/60))

the len of data is 100000
we use time 8.26547524134318 min


In [6]:
model

In [7]:
word2idx = model.wv.key_to_index
idx2word = model.wv.index_to_key
embeddingmatrix = []
for i,v in enumerate(idx2word):
    embeddingmatrix.append(model.wv[v])
#"PAD" vector
vec1 = np.random.rand(250)
#"UNK" vector
vec2 = np.random.rand(250)
word2idx["PAD"] = len(word2idx)
idx2word.append("PAD")
embeddingmatrix.append(vec1)
word2idx["UNK"] = len(word2idx)
idx2word.append("UNK")
embeddingmatrix.append(vec2)
#embeddingmatrix = torch.cat([embeddingmatrix,vec1,vec2],0)

In [8]:
# set the length of word is 40
senlen = 100
train_num = []
for senten in trlist:
    wordlist = []
    if len(senten) >= senlen:
        for sen in senten[:senlen]:
            if sen in word2idx.keys():
                wordlist.append(word2idx[sen])
            else:
                wordlist.append(word2idx['UNK'])
        train_num.append(wordlist)
    else:
        less_num = senlen - len(senten)
        for sen in senten:
            if sen in word2idx.keys():
                wordlist.append(word2idx[sen])
            else:
                wordlist.append(word2idx['UNK'])
        for _ in range(less_num):
            wordlist.append(word2idx['PAD'])
        train_num.append(wordlist)

In [9]:
import torch
from torch.utils import data
# define tordata type 自定义数据集类
class moviedata(data.Dataset):
    def __init__(self, X, y):
        self.data = X
        self.label = y
    def __getitem__(self, idx):
        if self.label is None: return self.data[idx]
        return self.data[idx], self.label[idx]
    def __len__(self):
        return len(self.data)

In [10]:
from torch import nn
from torch.autograd import Variable
class gru_net(nn.Module):
    def __init__(self,embedding, hidden_dim, num_layers, dropout=0.5,embedding_train=False):
        super(gru_net, self).__init__()
#         embedding = torch.Tensor(len(list(word_vectors)), len(word_vectors[0]))
#         word2vector_list = list(word_vectors)
#         for i in range(len(word2vector_list)-1):
#             get_vectors = word2vector_list[i]
#             embedding[i] = get_vectors
#         self.embedding = nn.Embedding.from_pretrained(embedding)
        self.embedding = torch.nn.Embedding(num_embeddings=len(embedding), embedding_dim=len(embedding[0]))
        self.embedding.weight.data.copy_(torch.Tensor(embedding.float()))
        self.embedding.weight.requires_grad = False
        
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.gru = nn.GRU(len(embedding[0]),self.hidden_dim,num_layers=self.num_layers,batch_first=True)
        
        self.classfier = nn.Sequential(
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_dim,1),
            nn.Sigmoid()
        )
    def forward(self,inputs):
        inputs = self.embedding(inputs.cuda())
        inputs = inputs.float().cuda()
        inputs = Variable(inputs,requires_grad = True)
        x, _ = self.gru(inputs, None)
        x = x[:,-1,:]# x is [batch,seq_len,hidden_size]
        x = self.classfier(x)
        return x

In [11]:
def evaluation(outputs, labels):
    # outputs => probability (float)
    # labels => labels
    outputs[outputs>=0.5] = 1 # 大於等於 0.5 為有惡意
    outputs[outputs<0.5] = 0 # 小於 0.5 為無惡意
    correct = torch.sum(torch.eq(outputs, labels)).item()
    return correct

In [12]:
import os
import torch
import argparse
import numpy as np
from torch import nn
from gensim.models import word2vec
import torch.optim as optim
#from sklearn.model_selection import train_test_split


In [25]:
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
best_valid_loss = float('inf') # 保存最小的loss
print(best_valid_loss)
def train_process(batch_size,epoch,lr,model_dir,train_loader,val_loader,gmodel,device,criterion):
    global best_valid_loss
    for i in range(epoch):
        train_loss = 0
        train_correct = 0
        val_loss = 0
        val_correct = 0
        for batch in train_loader:
            optimizer = optim.Adam(gmodel.parameters(), lr)
            optimizer.zero_grad()
            X, labels = batch
            labels.cuda()
            pred = gmodel(X).squeeze(1)
            loss = criterion(pred, labels.float().cuda())
            loss.backward()
            optimizer.step()# 更新权重
            train_loss += loss.item()
            train_correct += evaluation(pred, labels.cuda())
        with torch.no_grad():
            for batch in val_loader:
                X, labels = batch
                pred = gmodel(X).squeeze(1)
                loss = criterion(pred, labels.float().cuda())
                val_loss += loss.item()
                val_correct += evaluation(pred,labels.cuda())
            if val_loss < best_valid_loss :
                best_valid_loss = val_loss
                torch.save(gmodel.state_dict(), './gmodel.pt')
                
        print("epoch:",i,"train_loss:",train_loss,"train_correct:",train_correct)
        print("epoch:",i,"val_loss:",val_loss,"val_correct:",val_correct)

inf


In [53]:
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedding_train=False # fix embedding during training
batch_size = 256
epoch = 100
lr = 0.002
model_dir = './'
criterion = nn.BCEWithLogitsLoss() 
criterion = criterion.to(device)
print(len(train_num))
X = torch.LongTensor(train_num)
y = [int(rate) for rate in trrate]
y = torch.LongTensor(y)
embeddingmatrix = torch.from_numpy(np.array(embeddingmatrix))
#embeddingmatrix = torch.tensor(embeddingmatrix)
#establish model
gmodel = gru_net(embeddingmatrix, hidden_dim=150, num_layers=1, dropout=0.5,embedding_train=embedding_train)
gmodel = gmodel.to(device)
# split data
x_train,x_valid,y_train,y_valid = X[:18000],X[18000:],y[:18000],y[18000:]
# dataloader
train_dataset = moviedata(X=x_train, y=y_train)
val_dataset = moviedata(X=x_valid, y=y_valid)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = batch_size,
                                            shuffle = True,
                                            num_workers = 4)

val_loader = torch.utils.data.DataLoader(dataset = val_dataset,
                                            batch_size = batch_size,
                                            shuffle = False,
                                            num_workers = 4)
torch.cuda.empty_cache()
train_process(batch_size, epoch, lr, model_dir, train_loader, val_loader, gmodel, device,criterion)

test_data = []
senlen = 100
for sen in telist:
    wordlist=[]
    if len(sen) >= senlen:
        for word in sen[:senlen]:
            if word in word2idx.keys():
                wordlist.append(word2idx[word])
            else:
                wordlist.append(word2idx['UNK'])
        test_data.append(wordlist)
    else:
        for word in sen[:len(sen)]:
            if word in word2idx.keys():
                wordlist.append(word2idx[word])
            else:
                wordlist.append(word2idx['UNK'])
        for i in range(senlen - len(sen)) :
            wordlist.append(word2idx['PAD'])
        test_data.append(wordlist)
test_data = torch.LongTensor(test_data)
gmodel.eval()
df = pd.read_csv('./testData.tsv',quoting = 3,delimiter='\t')
sample = df[['id']]
result = []
with torch.no_grad():
    for X in test_data:
        pred = gmodel(X.unsqueeze(0))
        if pred >= 0.5:
            pred = 1
        else:
            pred = 0
        result.append(pred)
sample['sentiment'] = result
sample.to_csv('gru_model.csv',
             index=False, quoting=3)
        
    

25000
epoch: 0 train_loss: 47.97110956907272 train_correct: 10469
epoch: 0 val_loss: 17.548724949359894 val_correct: 4920
epoch: 1 train_loss: 44.86416482925415 train_correct: 13182
epoch: 1 val_loss: 18.33252340555191 val_correct: 5144
epoch: 2 train_loss: 43.55146384239197 train_correct: 13847
epoch: 2 val_loss: 16.95937007665634 val_correct: 5434
epoch: 3 train_loss: 43.162538290023804 train_correct: 14018
epoch: 3 val_loss: 17.150950968265533 val_correct: 5076
epoch: 4 train_loss: 42.72477573156357 train_correct: 14078
epoch: 4 val_loss: 16.773990869522095 val_correct: 5596
epoch: 5 train_loss: 42.335575580596924 train_correct: 14397
epoch: 5 val_loss: 16.595870554447174 val_correct: 5646
epoch: 6 train_loss: 41.888813614845276 train_correct: 14665
epoch: 6 val_loss: 16.76060599088669 val_correct: 5649
epoch: 7 train_loss: 41.71377170085907 train_correct: 14708
epoch: 7 val_loss: 16.892637610435486 val_correct: 5254
epoch: 8 train_loss: 41.395682752132416 train_correct: 14936
epoch